In [89]:
# !pip install pytorch-lightning

In [3]:
import torch
import os
from torch import optim, nn, utils, Tensor
import pytorch_lightning as pl
import numpy as np
from torch.optim import Adam

In [4]:
def arctan(x, x0, k):
    m = 0.5*(2/np.pi)*np.arctan(k*(x)+x0) + 0.5
    return m

# def GD(x, x0, k):
#     m = (4/np.pi)*np.arctan(np.tanh(k*(x-x0)))
#     return m

# def ERF(x, x0, k):
#     m = erf(k*(x-x0))
#     return m

# # def algebra(x, x0, k):
# #     m = x/((1+abs(x)**k)**(1/k))
# #     return m

# ### 3.7 meeting.  Weighted_combination of arctan+GD
# def arctan_GD(x,x0,k, w):
#     m = w*GD(x,x0,k)+(1-w)*arctan(x,x0,k)
#     return m

In [30]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.special import erf
from sklearn.preprocessing import MinMaxScaler
from matplotlib.ticker import FormatStrFormatter
from sklearn.metrics import mean_absolute_error as MSE
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import scipy as sp
import scipy
from scipy.optimize import curve_fit
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings('ignore')

In [7]:
def data_location(main_path, data, proteins):
  array = []
  for protein in proteins:
    array.append(str(main_path+data+protein))
  return array

In [122]:
DESIRED_SPECIES = "EC"
PATH = "/content/drive/MyDrive/random stuff/Adaptable-Sigmoids/data/"
PATH_truncate = len(PATH)

classes = ['ER', 'ERDD', 'GEO', 'GEOGD',  'SF', 'SFDD','Sticky'] #'Original''HGG',
# data_locations = [r"/content/drive/MyDrive/Adaptable-Sigmoids/data/AT"+c for c in classes]
data_locations = data_location(PATH, DESIRED_SPECIES, classes)

In [120]:
def combine_data(data_location):
  df_comb = pd.DataFrame()
  for protein in data_location:
    df = pd.read_csv(protein, header = None, sep = ' ')
    df['class'] = protein[PATH_truncate:]
    df_comb = pd.concat([df, df_comb])
  return df_comb

In [123]:
df_comb = combine_data(data_locations)
df_class = df_comb['class']
df_comb = df_comb.drop("class", axis = 1)
df_comb = pd.DataFrame(MinMaxScaler().fit_transform(df_comb))
df_comb['class'] = df_class.reset_index(drop = True)
df_comb

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,class
0,0.263271,0.903052,0.098359,0.935637,0.482611,0.387326,0.623361,0.059294,0.002245,0.920228,...,0.425188,0.055862,0.021093,5.047120e-02,0.118947,0.003169,0.021723,0.000329,0.000019,ECSticky
1,0.140483,0.867787,0.092337,0.891363,0.441625,0.358539,0.560003,0.051199,0.001722,0.871713,...,0.367881,0.043296,0.015911,4.022777e-02,0.099554,0.002462,0.018028,0.000253,0.000009,ECSticky
2,0.157105,0.878349,0.092507,0.899562,0.460213,0.361891,0.573181,0.052723,0.001890,0.876664,...,0.377219,0.048787,0.017813,4.364059e-02,0.105748,0.002754,0.019372,0.000293,0.000006,ECSticky
3,0.258445,0.889638,0.090449,0.918930,0.453742,0.372525,0.553283,0.048341,0.001631,0.903812,...,0.372902,0.037491,0.015036,3.668705e-02,0.098968,0.002130,0.015991,0.000205,0.000006,ECSticky
4,0.185523,0.884801,0.092054,0.908156,0.458550,0.366616,0.561481,0.049014,0.001572,0.891801,...,0.368222,0.038201,0.014338,3.713699e-02,0.096485,0.002025,0.014934,0.000187,0.000003,ECSticky
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3487,0.149598,0.257404,0.001114,0.139349,0.045183,0.002132,0.002112,0.000033,0.000000,0.072684,...,0.000097,0.000000,0.000000,9.289976e-07,0.000000,0.000000,0.000000,0.000000,0.000000,ECER
3488,0.112064,0.252945,0.001322,0.136739,0.044350,0.002506,0.002002,0.000050,0.000000,0.071086,...,0.000099,0.000001,0.000000,2.167661e-06,0.000007,0.000000,0.000000,0.000000,0.000000,ECER
3489,0.216086,0.264572,0.001322,0.143245,0.046045,0.003091,0.002368,0.000041,0.000000,0.075023,...,0.000099,0.000001,0.000000,1.238664e-06,0.000007,0.000000,0.000000,0.000000,0.000000,ECER
3490,0.277748,0.273776,0.001529,0.149217,0.048015,0.003173,0.002766,0.000033,0.000000,0.078880,...,0.000134,0.000000,0.000000,1.857995e-06,0.000014,0.000000,0.000000,0.000000,0.000000,ECER


In [124]:
x = df_comb.iloc[0:,0:30]
x_train, x_test, y_train, y_test = train_test_split(x, df_comb['class'])

In [125]:
Y_train = pd.get_dummies(y_train, prefix = ['class'])
Y_train = Y_train.set_axis([df_comb['class'].unique()], axis = 1)
Y_test = pd.get_dummies(y_test, prefix = ['class'])
Y_test = Y_test.set_axis([df_comb['class'].unique()], axis = 1)

In [131]:
ECER = LogisticRegression().fit(x_train,Y_train['ECER'])
ECERDD = LogisticRegression().fit(x_train,Y_train['ECERDD'])
ECGEO = LogisticRegression().fit(x_train,Y_train['ECGEO'])
ECGEOGD = LogisticRegression().fit(x_train,Y_train['ECGEOGD'])
ECSF = LogisticRegression().fit(x_train,Y_train['ECSF'])
ECSFDD = LogisticRegression().fit(x_train,Y_train['ECSFDD'])
ECSticky = LogisticRegression().fit(x_train,Y_train['ECSticky'])

logit_classes = [ECER, ECERDD, ECGEO, ECGEOGD, ECSF, ECSFDD, ECSticky]#, ECERDD, ECGEO, ECGEOGD, ECSF, ECSFDD, ECSTicky]
classes = np.flip(df_comb['class'].unique())
for logit,class_name in zip(logit_classes, classes):
  print(logit.score(x_test, Y_test[f'{class_name}']))
  # print(logit, class_name)


1.0
0.997709049255441
1.0
0.9965635738831615
1.0
0.9988545246277205
1.0


In [127]:
orig_protein = pd.read_csv(f'{PATH}' + DESIRED_SPECIES + 'Original', header = None, sep = ' ')
orig_protein = MinMaxScaler().fit_transform(orig_protein)
orig_protein

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [128]:
print("#############probabilities#############\n")


for logit,class_name in zip(logit_classes, classes):
  # print(logit.predict_proba(orig_protein))
  print(class_name, logit.predict_proba(orig_protein)[0][1])

print("\n#############PREDICTIONS#############\n")

for logit,class_name in zip(logit_classes, classes):
  print(class_name, logit.predict(orig_protein))
  # print(logit.predict_proba(orig_protein)[0][1])

#############probabilities#############

ECER 0.0008497095614894864
ECERDD 0.025189374367080887
ECGEO 0.0004454331866046466
ECGEOGD 0.0005535671015546483
ECSF 0.33508647751389725
ECSFDD 6.821995247907611e-05
ECSticky 0.9623323598615613

#############PREDICTIONS#############

ECER [0]
ECERDD [0]
ECGEO [0]
ECGEOGD [0]
ECSF [0]
ECSFDD [0]
ECSticky [1]


In [129]:
# class MyDataset(Dataset):
 
#   def __init__(self,file_name):
#     price_df=pd.read_csv(file_name)
 
#     x=price_df.iloc[:,0:8].values
#     y=price_df.iloc[:,8].values
 
#     self.x_train=torch.tensor(x,dtype=torch.float32)
#     self.y_train=torch.tensor(y,dtype=torch.float32)
 
#   def __len__(self):
#     return len(self.y_train)
   
#   def __getitem__(self,idx):
#     return self.x_train[idx],self.y_train[idx]

In [130]:
# class SigmoidModel(pl.LightningModule):
#   def __init__(self, input_dim, output_dim = 1):
#     super().__init__()
#     self.fc1 = nn.Linear(input_dim, 1)

#   def sigmoid(self, x, w, b):
#     m = 0.5*(2/np.pi)*np.arctan(x*w+b) + 0.5
#     return m
    
# #TODO: tidy up with nn.Sequential
#   def forward(self, x):
#     logits = self.sigmoid(self.fc1(x))
#     return logits

#   def configure_optimizers(self):
#     return Adam(self.parameters(),lr = self.lr)

#   def training_step(self, train_batch, batch_idx):
#     x, y = train_batch
#     logits = self.forward(x)
#     loss = self.loss(logits, y)
#     return loss

#   def validation_step(self, valid_batch, batch_idx):
#     x, y = valid_batch
#     logits = self.forward(x)
#     loss = self.loss(logits,y)

